# IMPORT & SET UP

In [563]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import urllib.parse
import os
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_colwidth', None)

In [564]:
PROJECT_DIR = os.getcwd()
ID_DIR_NAME = 'job_id_dir'
INFO_DIR_NAME = 'job_info_dir'

### DRIVER FUNCTIONS

In [565]:
DRIVER = None

In [566]:
def start_driver():
    global DRIVER
    if DRIVER is None:
        options = Options()
        options.add_argument('--headless=new')
        options.add_argument('--no-sandbox')
        options.add_argument('--incognito')
        options.add_argument('-disable-dev-shm-usage')

        chromedriver_path = PROJECT_DIR + 'Chromedriver/chromedriver_mac_arm64/chromedriver'
        service = Service(chromedriver_path)
        DRIVER = webdriver.Chrome(service=service, options=options)
    else:
        print('A driver is already running. Please close current driver before starting another one.')

def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
        DRIVER = None

In [567]:
close_driver()

In [568]:
start_driver()

### URL & LOGIN SETUPS

In [569]:
from linkedin_scraper import actions

EMAIL = "minhnnt97@gmail.com"
PASSWORD = "pazqig-higdar-nehmE6"
actions.login(DRIVER, EMAIL, PASSWORD) 

BASE_URL = "https://www.linkedin.com/jobs/search/"
SEARCH_TERM = "Data Scientist"
LOCATION_TERM = "United States"

SEARCH_URL = BASE_URL + f"?keywords={urllib.parse.quote(SEARCH_TERM)}" + f"&location={urllib.parse.quote(LOCATION_TERM)}&refresh=true"
SEARCH_URL

'https://www.linkedin.com/jobs/search/?keywords=Data%20Scientist&location=United%20States&refresh=true'

# FUNCTIONS

## Function to scrape all job IDs on the search page(s)

In [570]:
# Function to scrape all products from a page
def get_all_job_ids_from_page(url, num_page=-1):

    global DRIVER, SEARCH_TERM, LOCATION_TERM

    job_id_scrape_time = datetime.now()
    print(f'DATE & TIME: {job_id_scrape_time.strftime(("%Y/%m/%d %H:%M:%S"))}')
    job_id_list = []      
    DRIVER.get(url) 
    time.sleep(5) 

      # Count total number of pages in the search result
    max_page = DRIVER.find_element(By.CLASS_NAME, 'jobs-search-results-list__pagination').find_element(By.TAG_NAME, 'ul').find_elements(By.TAG_NAME, 'li')[-1]
    max_page = int(max_page.text)
    num_page = min(max_page, num_page) if num_page>0 else max_page
    print(f'Scraping {num_page} page(s) out of {max_page} total pages for {SEARCH_TERM} jobs search in {LOCATION_TERM}...')
    total_err_count = 0

      # Loop through num_page pages to scrape all products listed on each page
    for p in range(1, num_page+1):
      url = url + f'&start={(p-1)*25}'  # p=1: start=0, p=2: start = 25 ...
      DRIVER.get(url)
      time.sleep(3)
      # Find all li tags that contain the job information
      products_all = DRIVER.find_element(By.CLASS_NAME, "jobs-search-results-list").find_element(By.CLASS_NAME, "scaffold-layout__list-container").find_elements(By.CLASS_NAME, "scaffold-layout__list-item")
      print(f'Found {len(products_all)} job IDs on page {p}.', end=' ')

      err_count = 0
      for product in products_all:
          try:
              job_id = product.get_attribute("data-occludable-job-id")
              job_id_list.append(job_id)
          except NoSuchElementException:
              err_count += 1

      print(f'Could not scrape {err_count} jobs.')
      total_err_count += err_count

    # Remove dupes
    job_id_list = list(set(job_id_list))
    print(f'---> Found total {len(job_id_list)} unique jobs. Could not scrape total of {total_err_count} jobs.')

    return job_id_list, job_id_scrape_time


## Function to scrape information on the job page

In [574]:
def get_single_job_info(job_id):

    info = {
        'Job ID': job_id,
        'Job URL': None,
        'Name': None,
        'Company': None,
        'Company Logo URL': None,
        'Location': None,
        'Workplace Type': None,
        'Time Posted': None,
        'Applicants Count': None,
        'Job Overview': None,
        'Company Overview': None,
        'HR URL': None
    }

    JOB_URL = f"https://www.linkedin.com/jobs/view/{job_id}"
    DRIVER.get(JOB_URL)
    time.sleep(3)

    # Job URL
    info['Job URL'] = JOB_URL

    # Job Name
    try:
        name = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__job-title").get_attribute("innerHTML")
        info['Name'] = name.strip()
    except NoSuchElementException:
        pass

    # Company
    try:
        company = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__company-name").find_element(By.TAG_NAME, 'a').get_attribute("innerHTML")
        info['Company'] = company.strip()
    except NoSuchElementException:
        pass

    # Company Logo
    try:
        comp_logo_url = DRIVER.find_element(By.CLASS_NAME, 'p5').find_element(By.TAG_NAME, 'img').get_attribute('src')
        info['Company Logo URL'] = comp_logo_url.strip()
    except NoSuchElementException:
        pass

    # Location
    try:
        location = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__subtitle-primary-grouping").find_element(By.CLASS_NAME, "jobs-unified-top-card__bullet").get_attribute("innerHTML")
        info['Location'] = location.strip()
    except NoSuchElementException:
        pass

    # Workplace Type
    try:
        work_type = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__workplace-type").get_attribute("innerHTML")
        info['Workplace Type'] = work_type.strip()
    except NoSuchElementException:
        pass

    # Time Posted
    try:
        time_posted = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__posted-date").get_attribute("innerHTML")
        info['Time Posted'] = time_posted.strip()
    except NoSuchElementException:
        pass

    # Applicants Count
    try:
        applicants = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__subtitle-secondary-grouping").find_element(By.CLASS_NAME, "jobs-unified-top-card__applicant-count").get_attribute("innerHTML")
        info['Applicants Count'] = applicants.strip()
    except NoSuchElementException:
        try:
            applicants = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__subtitle-secondary-grouping").find_element(By.CLASS_NAME, "jobs-unified-top-card__bullet").get_attribute("innerHTML")
            info['Applicants Count'] = applicants.strip()
        except NoSuchElementException:
            pass

    # Job & Company Insight
    try:
        job_overview, company_overview = DRIVER.find_elements(By.CLASS_NAME, "jobs-unified-top-card__job-insight")[:2]
        # use try except to find info of each column:
        # try:
        #     salary = job_overview.find_element(By.TAG_NAME, 'span').find_element(By.TAG_NAME, 'a').get_attribute("innerHTML")
        #     info['Salary'] = salary.strip('\n<!-> ')
        #     job_overview = job_overview.find_element(By.TAG_NAME, 'span').text # With salary posted
        # except NoSuchElementException:
        #     job_overview = job_overview.find_element(By.TAG_NAME, 'span').get_attribute("innerHTML") # Without salary posted

        # use .text to get all text information
        job_overview = job_overview.find_element(By.TAG_NAME, 'span').text
        job_overview = re.sub(r'<!--(?=.*?-->).*?-->', '', job_overview, flags=re.DOTALL) # Remove all HTML comments

        company_overview = company_overview.find_element(By.TAG_NAME, 'span').text
        company_overview = re.sub(r'<!--(?=.*?-->).*?-->', '', company_overview, flags=re.DOTALL) # Remove all HTML comments

        info['Job Overview'] = job_overview.strip()
        info['Company Overview'] = company_overview.strip()
    except NoSuchElementException:
        pass

    # HR URL
    try:
        hr_url = DRIVER.find_element(By.CSS_SELECTOR, "div[class*='hirer-card__hirer-information'] a").get_attribute('href')
        info['HR URL'] = hr_url.strip()
    except NoSuchElementException:
        pass

    # Job details
    try:
        job_details = DRIVER.find_element(By.ID, 'job-details').find_element(By.TAG_NAME, 'span').text
        job_details = re.sub(r'<!--(?=.*?-->).*?-->', '', job_details, flags=re.DOTALL) # Remove all HTML comments
        info['Job Details'] = job_details.strip()
    except NoSuchElementException:
        pass

    return info


#### Test job_id

In [557]:
test_info = get_single_job_info(3525179725) 
test_info

{'Job ID': 3525179725,
 'Job URL': 'https://www.linkedin.com/jobs/view/3525179725',
 'Name': 'AI Research Engineer- Remote',
 'Company': 'Neo Cybernetica',
 'Company Logo URL': 'https://media.licdn.com/dms/image/C560BAQFdt_fuOIJ5TQ/company-logo_100_100/0/1644369586489?e=1688601600&v=beta&t=QyxXmGwyZyxqfDP7SiKcGuUJGrnmgnsnpdG5FbV-ESw',
 'Location': 'Bedford, NH',
 'Workplace Type': 'Remote',
 'Time Posted': '2 weeks ago',
 'Applicants Count': '81 applicants',
 'Job Overview': '11-50 employees · Software Development',
 'Company Overview': 'See how you compare to 81 applicants. Try Premium for free',
 'HR URL': None,
 'Job Details': 'About Us\n\nWe are a next-gen cybernetics start-up backed by a few top-tier investors (led by NEA).\n\nOur R&D blends robotics, machine learning, and high-fidelity simulation. We aim to push the boundaries of what intelligent systems are capable of achieving both autonomously and in collaboration with humans.\n\nBefore starting Neo Cybernetica, our CEO founde

In [572]:
test_info = get_single_job_info(3527055540)
test_info

{'Job ID': 3527055540,
 'Job URL': 'https://www.linkedin.com/jobs/view/3527055540',
 'Name': 'Data Analyst',
 'Company': 'Fenway Health',
 'Company Logo URL': 'https://media.licdn.com/dms/image/C4D0BAQFA0rlVxLg0HA/company-logo_100_100/0/1519856202235?e=1688601600&v=beta&t=RbwZT1Nr218rM1CX3vxqV7L3_VY2xfsUCZy2ieT0rMM',
 'Location': 'Boston, MA',
 'Workplace Type': 'On-site',
 'Time Posted': '2 weeks ago',
 'Applicants Count': '155 applicants',
 'Job Overview': 'Part-time · Entry level',
 'Company Overview': '201-500 employees · Hospitals and Health Care',
 'HR URL': None,
 'Job Details': 'About the job\nReporting to the Director of Data Analytics, the Data Analyst I accurately provides data management support, data analyses, and data visualizations to internal and external stakeholders.\n\nRepresentative Duties\n\nQueries, analyzes, visualizes, and interprets data and serves as a content expert on existing data sources and data reporting tools'}

In [571]:
test_info = get_single_job_info(3551940981) # without .find_element(By.TAG_NAME, 'span') --> scrape partial job detail (text in ul is not scraped)
test_info

{'Job ID': 3551940981,
 'Job URL': 'https://www.linkedin.com/jobs/view/3551940981',
 'Name': 'Data Analyst',
 'Company': 'Harvard University',
 'Company Logo URL': 'https://media.licdn.com/dms/image/C4E0BAQF5t62bcL0e9g/company-logo_100_100/0/1519855919126?e=1688601600&v=beta&t=pAF1AvNhhJet9rY9oA43thyAPwGffcV8N7kQ644qj1k',
 'Location': 'Harvard, MA',
 'Workplace Type': 'On-site',
 'Time Posted': '15 hours ago',
 'Applicants Count': '5 applicants',
 'Job Overview': 'Temporary · Entry level',
 'Company Overview': '10,001+ employees · Higher Education',
 'HR URL': None,
 'Job Details': 'About the job\nThis job is sourced from a job board. Learn more\n61864BRAuto req ID:61864BRJob Code:I0857P IT Business Analysis Prfss III Department Office Location:USA - MA - Cambridge Business Title:Data AnalystSub-Unit:Division of Continuing Education Salary Grade (https://hr.harvard.edu/salary-ranges#ranges) :057Time Status:Full-time Union:00 - Non Union, Exempt or Temporary Additional Qualifications an

In [575]:
test_info = get_single_job_info(3551940981)  # with .find_element(By.TAG_NAME, 'span')
test_info

{'Job ID': 3551940981,
 'Job URL': 'https://www.linkedin.com/jobs/view/3551940981',
 'Name': 'Data Analyst',
 'Company': 'Harvard University',
 'Company Logo URL': 'https://media.licdn.com/dms/image/C4E0BAQF5t62bcL0e9g/company-logo_100_100/0/1519855919126?e=1688601600&v=beta&t=pAF1AvNhhJet9rY9oA43thyAPwGffcV8N7kQ644qj1k',
 'Location': 'Harvard, MA',
 'Workplace Type': 'On-site',
 'Time Posted': '15 hours ago',
 'Applicants Count': '5 applicants',
 'Job Overview': 'Temporary · Entry level',
 'Company Overview': '10,001+ employees · Higher Education',
 'HR URL': None,
 'Job Details': 'This job is sourced from a job board. Learn more'}

# START SCRAPING

Get all job ids from the job search page into a list

In [13]:
num_page = 0 # to scrape all pages: set to 0 or -1
job_id_list, job_id_scrape_time = get_all_job_ids_from_page(SEARCH_URL, num_page=num_page)
# job_id_list

DATE & TIME: 2023/03/29 13:10:29
Scraping 40 page(s) out of 40 total pages for Data Scientist jobs search in United States...
Found 25 job IDs on page 1. Could not scrape 0 jobs.
Found 25 job IDs on page 2. Could not scrape 0 jobs.
Found 25 job IDs on page 3. Could not scrape 0 jobs.
Found 25 job IDs on page 4. Could not scrape 0 jobs.
Found 25 job IDs on page 5. Could not scrape 0 jobs.
Found 25 job IDs on page 6. Could not scrape 0 jobs.
Found 25 job IDs on page 7. Could not scrape 0 jobs.
Found 25 job IDs on page 8. Could not scrape 0 jobs.
Found 25 job IDs on page 9. Could not scrape 0 jobs.
Found 25 job IDs on page 10. Could not scrape 0 jobs.
Found 25 job IDs on page 11. Could not scrape 0 jobs.
Found 25 job IDs on page 12. Could not scrape 0 jobs.
Found 25 job IDs on page 13. Could not scrape 0 jobs.
Found 25 job IDs on page 14. Could not scrape 0 jobs.
Found 25 job IDs on page 15. Could not scrape 0 jobs.
Found 25 job IDs on page 16. Could not scrape 0 jobs.
Found 25 job IDs on

#### Write list of job ids to file

In [436]:
file_name = f'jobs_{job_id_scrape_time.strftime("%y%m%d_%H%M%S")}.txt'
JOB_FILE_PATH = os.path.join(PROJECT_DIR, ID_DIR_NAME, file_name)
print(f'[{job_id_scrape_time.strftime(r"%Y/%m/%d %H:%M:%S")}] Updated job list at {JOB_FILE_PATH}')

# Write file to folder
# with open(JOB_FILE_PATH, 'w+') as f:
#     for job_id in job_id_list:
#         f.write(f'{job_id}\n')

[2023/03/29 13:10:29] Updated job list at /Users/thule/Desktop/DSProjects/Linkedin_Analysis/job_id_list_dir/jobs_230329_131029.txt


#### Get list of job id from most recent file

In [279]:
# Get most recent id file
JOB_FILE_PATH = os.path.join(PROJECT_DIR, ID_DIR_NAME, (os.listdir(os.path.join(PROJECT_DIR, ID_DIR_NAME)))[-1])

with open(JOB_FILE_PATH, 'r') as f:
    job_id_list = [l.strip() for l in f.readlines()]

job_id_list

['3520044004',
 '3523743282',
 '3520459813',
 '3525723554',
 '3527821880',
 '3545964641',
 '3520729278',
 '3516637194',
 '3527065467',
 '3522212905',
 '3522294979',
 '3529041711',
 '3532901532',
 '3529458696',
 '3516672427',
 '3529872684',
 '3529949220',
 '3525654696',
 '3525923153',
 '3520439276',
 '3520458644',
 '3524955715',
 '3530936787',
 '3521523630',
 '3522086390',
 '3521754879',
 '3517168914',
 '3542549318',
 '3522010736',
 '3545964865',
 '3533040991',
 '3526783948',
 '3525790897',
 '3523779032',
 '3522879882',
 '3525207467',
 '3520737436',
 '3527860425',
 '3514747876',
 '3531333875',
 '3532789301',
 '3529457989',
 '3516670537',
 '3520378880',
 '3545989312',
 '3520382049',
 '3520155411',
 '3528643036',
 '3516616194',
 '3526077070',
 '3520820840',
 '3528020320',
 '3531482679',
 '3524248732',
 '3532595880',
 '3540122981',
 '3522671237',
 '3530768301',
 '3531339677',
 '3521702352',
 '3532599805',
 '3527675015',
 '3525729542',
 '3528295490',
 '3542540251',
 '3525564609',
 '35280216

Get information from all job page

In [ ]:
from tqdm.notebook import tqdm

job_info_list = []

for job_id in tqdm(job_id_list):
    try:
        job_info = get_single_job_info(job_id)
        job_info_list.append(job_info)
    except Exception as e:
        print(e)

job_info_list[:3]

In [281]:
job_df = pd.DataFrame(job_info_list)
job_df.head(10)

""


#### Save DataFrame to csv file

In [ ]:
JOB_INFO_PATH = os.path.join(PROJECT_DIR, INFO_DIR_NAME, os.path.basename(JOB_FILE_PATH)[:-3] + 'csv')

job_df.to_csv(JOB_INFO_PATH, index=0, sep='@')

#### Load most recent DataFrame from csv file

In [283]:
JOB_INFO_PATH = os.path.join(PROJECT_DIR, INFO_DIR_NAME, (os.listdir(os.path.join(PROJECT_DIR, INFO_DIR_NAME)))[-1])

job_df = pd.read_csv(JOB_INFO_PATH, index_col=0, sep='@')
job_df.head(5)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview
0,3520044004,https://www.linkedin.com/jobs/view/3520044004,"Data Scientist, Research",TikTok,https://media.licdn.com/dms/image/C510BAQGCdThXIss7UQ/company-logo_100_100/0/1539940587971?e=1687996800&v=beta&t=1AyKLKEh0SiRwHFVslSVLJ-5LaOWsCzFqZtQ9ZvHi1o,"San Jose, CA",Hybrid,2 weeks ago,198 applicants,Full-time,"10,001+ employees · Entertainment Providers"
1,3523743282,https://www.linkedin.com/jobs/view/3523743282,Global Data Scientist,Kimberly-Clark,https://media.licdn.com/dms/image/C560BAQFahtjOdf_ETQ/company-logo_100_100/0/1542208571146?e=1687996800&v=beta&t=y-aRHU6gnrNyr6nMPswJJHlWEoEOjZyHL-a1Qs5MPFY,United States,Remote,2 weeks ago,Over 200 applicants,Full-time · Mid-Senior level,"10,001+ employees · Manufacturing"
2,3520459813,https://www.linkedin.com/jobs/view/3520459813,Data Analytics Intern (Summer 2023),Industry Dive,https://media.licdn.com/dms/image/C4E0BAQEAkpLAgFUtpA/company-logo_100_100/0/1520994058492?e=1687996800&v=beta&t=wkHpT8VLjHRZryS_VDdpboOGmMkNRTNhMqVoDrCC2_M,"Washington, DC",Remote,NaN,NaN,$16/hr - $21/hr (from job description) · Internship · Internship,201-500 employees · Online Audio and Video Media
3,3525723554,https://www.linkedin.com/jobs/view/3525723554,Data Scientist Solution Specialist- IT Internship,Waters Corporation,https://media.licdn.com/dms/image/C560BAQHFDhBFVWfhzg/company-logo_100_100/0/1656651227123?e=1687996800&v=beta&t=YmLFUIyNIljy5hv0bST7P3wzrzdVBRln9qUpFAufG0c,"Milford, MA",On-site,1 week ago,Over 200 applicants,Internship · Internship,"5,001-10,000 employees · Biotechnology Research"
4,3527821880,https://www.linkedin.com/jobs/view/3527821880,Data Engineer,Chatham Financial,https://media.licdn.com/dms/image/C4D0BAQFPJJtAqKZSKA/company-logo_100_100/0/1566565993951?e=1687996800&v=beta&t=w4VlO0akxyqbbvH7Io6cMb3i8qShWG84zirwSGF-rDM,"Kennett Square, PA",On-site,2 weeks ago,104 applicants,Full-time · Entry level,"501-1,000 employees · Financial Services"


# CLEANING

In [284]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 885 entries, 0 to 884
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Job ID            885 non-null    int64 
 1   Job URL           885 non-null    object
 2   Name              885 non-null    object
 3   Company           883 non-null    object
 4   Company Logo URL  885 non-null    object
 5   Location          885 non-null    object
 6   Workplace Type    754 non-null    object
 7   Time Posted       778 non-null    object
 8   Applicants Count  776 non-null    object
 9   Job Overview      885 non-null    object
 10  Company Overview  885 non-null    object
dtypes: int64(1), object(10)
memory usage: 83.0+ KB


In [285]:
job_df_2 = job_df.copy() # Make a copy

### Split Location
New columns:
- Location_City
- Location_State

In [508]:
job_df['Location'].str.split(', ').map(len).value_counts()

2    776
1    104
3      5
Name: Location, dtype: int64

#### Locations that have 3 values:

- City, States, Country

In [509]:
idx = job_df[job_df['Location'].str.split(', ').map(len)==3].index.to_list()
job_df[['Location_City', 'Location_State', 'Location_Country']] = job_df.iloc[idx]['Location'].str.split(', ', expand=True)
job_df.iloc[idx,  5:]

,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Location_Country
158,"Los Angeles, California, United States",Remote,NaN,NaN,"$130,000/yr - $180,000/yr (from job description) · Full-time · Mid-Senior level","501-1,000 employees · Staffing and Recruiting",Los Angeles,California,United States
460,"Fort Bragg, North Carolina, United States",On-site,2 weeks ago,76 applicants,Full-time · Entry level,51-200 employees · IT Services and IT Consulting,Fort Bragg,North Carolina,United States
477,"Fort Bragg, North Carolina, United States",NaN,2 weeks ago,1 applicant,Full-time · Entry level,11-50 employees · Defense and Space Manufacturing,Fort Bragg,North Carolina,United States
577,"La Jolla, California, United States",On-site,2 weeks ago,5 applicants,Contract · Entry level,"1,001-5,000 employees · Staffing and Recruiting",La Jolla,California,United States
691,"New York, New York, United States",NaN,1 week ago,45 applicants,"$105,000/yr - $116,000/yr (from job description) · Full-time · Mid-Senior level","10,001+ employees · Telecommunications",New York,New York,United States


#### Locations that have 2 values:
- City, State
- State, Country

In [510]:
pattern_1 = r'(.+), (United States)'
pattern_2 = r'(.+), ([A-Z]{2})'
mask = (job_df['Location'].str.split(', ').map(len)==2)

for idx, row in job_df[mask].iterrows():
    seq = row['Location']

    location_1 = re.search(pattern_1, seq)
    location_2 = re.search(pattern_2, seq)

    if location_1:
        job_df.at[idx, 'Location_State'] = location_1.group(1)
    elif location_2:
        job_df.at[idx, 'Location_City'] = location_2.group(1)
        job_df.at[idx, 'Location_State'] = location_2.group(2)
    else:
        'Unknown'

job_df.iloc[10:15, 5:]

,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Location_Country
10,"Washington, DC",NaN,2 weeks ago,Over 200 applicants,Contract · Entry level,201-500 employees · Information Technology &amp; Services,Washington,DC,NaN
11,"Tampa, FL",Hybrid,2 weeks ago,83 applicants,Full-time · Mid-Senior level,"10,001+ employees · IT Services and IT Consulting",Tampa,FL,NaN
12,United States,Remote,1 week ago,Over 200 applicants,"$72,800/yr - $109,200/yr (from job description) · Full-time · Entry level",201-500 employees · Education Management,NaN,NaN,NaN
13,"Maine, United States",Hybrid,2 weeks ago,38 applicants,Full-time · Mid-Senior level,"501-1,000 employees · Technology, Information and Internet",NaN,Maine,NaN
14,"McLean, VA",Hybrid,2 weeks ago,113 applicants,"$58,400/yr - $133,000/yr (from job description) · Full-time","10,001+ employees · IT Services and IT Consulting",McLean,VA,NaN


### Except Washington DC

#### Locations that have only 1 information

In [511]:
job_df[~(job_df['Location'].str.contains(r', '))]['Location'].value_counts()

United States                          69
New York City Metropolitan Area         9
San Francisco Bay Area                  6
Washington DC-Baltimore Area            3
Greater Boston                          3
Cincinnati Metropolitan Area            2
Greater Chicago Area                    2
Des Moines Metropolitan Area            2
Dallas-Fort Worth Metroplex             2
Atlanta Metropolitan Area               1
Greater Sacramento                      1
Greater Houston                         1
Greater Vancouver Metropolitan Area     1
Greater Seattle Area                    1
Los Angeles Metropolitan Area           1
Name: Location, dtype: int64

In [512]:
mask = ~(job_df['Location'].str.contains(r', ')) & (job_df['Location']!='United States')
job_df.loc[mask, 'Location_City'] = job_df[mask]['Location']
job_df[mask].iloc[:3,5:]

,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Location_Country
39,New York City Metropolitan Area,Hybrid,1 week ago,185 applicants,Contract,"10,001+ employees · Beverage Manufacturing",New York City Metropolitan Area,NaN,NaN
56,Washington DC-Baltimore Area,Hybrid,2 weeks ago,33 applicants,"$85,000/yr · Full-time",1-10 employees,Washington DC-Baltimore Area,NaN,NaN
117,Cincinnati Metropolitan Area,Hybrid,1 day ago,Over 200 applicants,Contract · Mid-Senior level,"501-1,000 employees · IT Services and IT Consulting",Cincinnati Metropolitan Area,NaN,NaN


### Match State Names with Abbreviation

In [513]:
# remove city col for jobs in Washington, DC
job_df.loc[job_df['Location_City']=='Washington', 'Location_City'] = None

In [514]:
states_dict = {
'Alabama':	            'AL',
'Kentucky':	            'KY',
'Ohio':	                'OH',
'Alaska':	            'AK',
'Louisiana':	        'LA',
'Oklahoma':	            'OK',
'Arizona':	            'AZ',
'Maine':	            'ME',
'Oregon':	            'OR',
'Arkansas':	            'AR',
'Maryland':	            'MD',
'Pennsylvania':	        'PA',
'American Samoa':	    'AS',
'Massachusetts':	    'MA',
'Puerto Rico':	        'PR',
'California':	        'CA',
'Michigan':	            'MI',
'Rhode Island':	        'RI',
'Colorado':	            'CO',
'Minnesota':	        'MN',
'South Carolina':	    'SC',
'Connecticut':	        'CT',
'Mississippi':	        'MS',
'South Dakota':	        'SD',
'Delaware':	            'DE',
'Missouri':	            'MO',
'Tennessee':	        'TN',
'District of Columbia':	'DC',	
'Montana':	            'MT',
'Texas':	            'TX',
'Florida':	            'FL',
'Nebraska':	            'NE',
'Trust Territories':	'TT',
'Georgia':	            'GA',
'Nevada':	            'NV',
'Utah':	                'UT',
'Guam':	                'GU',
'New Hampshire':	    'NH',
'Vermont':	            'VT',
'Hawaii':	            'HI',
'New Jersey':	        'NJ',
'Virginia':	            'VA',
'Idaho':	            'ID',
'New Mexico':	        'NM',
'Virgin Islands':	    'VI',
'Illinois':	            'IL',
'New York':	            'NY',
'Washington':	        'WA',
'Indiana':	            'IN',
'North Carolina':	    'NC',
'West Virginia':	    'WV',
'Iowa':	                'IA',
'North Dakota':	        'ND',
'Wisconsin':	        'WI',
'Kansas':	            'KS',
'Northern Mariana Islands':	'MP',	
'Wyoming':	            'WY'
}
states_dict = dict(sorted(states_dict.items()))
states_dict = dict((v, k) for k, v in states_dict.items())
states_dict

{'AL': 'Alabama',
 'AK': 'Alaska',
 'AS': 'American Samoa',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'DC': 'District of Columbia',
 'FL': 'Florida',
 'GA': 'Georgia',
 'GU': 'Guam',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'MP': 'Northern Mariana Islands',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'PR': 'Puerto Rico',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'TT': 'Trust Territories',
 'UT': 

In [515]:
job_df['Location_State'] = job_df['Location_State'].apply(lambda x: states_dict[x] if x in states_dict else x)
job_df['Location_State'].unique()

array(['California', nan, 'District of Columbia', 'Massachusetts',
       'Pennsylvania', 'New Jersey', 'Illinois', 'Virginia', 'Florida',
       'Maine', 'Kansas', 'Texas', 'Colorado', 'Nevada', 'Arkansas',
       'Georgia', 'Maryland', 'New York', 'Delaware', 'Washington',
       'North Carolina', 'Connecticut', 'Rhode Island', 'Missouri',
       'South Carolina', 'Indiana', 'Iowa', 'Ohio', 'Hawaii',
       'New Hampshire', 'Alabama', 'Wisconsin', 'Oklahoma', 'Tennessee',
       'Michigan', 'Kentucky', 'Oregon', 'Idaho', 'Minnesota', 'Vermont',
       'Nebraska', 'Arizona', 'South Dakota', 'Louisiana', 'Utah',
       'Mississippi', 'New Mexico', 'Montana'], dtype=object)

In [516]:
job_df = job_df.drop(columns=['Location_Country'])

#### Fill 'Remote' location for Remote jobs

In [517]:
job_df.loc[(job_df['Workplace Type']=='Remote'), 'Location_City'] = 'Remote'
job_df.loc[(job_df['Workplace Type']=='Remote'), 'Location_State'] = 'Remote'

In [518]:
job_df.iloc[10:15,5:] # Recheck

,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State
10,"Washington, DC",NaN,2 weeks ago,Over 200 applicants,Contract · Entry level,201-500 employees · Information Technology &amp; Services,None,District of Columbia
11,"Tampa, FL",Hybrid,2 weeks ago,83 applicants,Full-time · Mid-Senior level,"10,001+ employees · IT Services and IT Consulting",Tampa,Florida
12,United States,Remote,1 week ago,Over 200 applicants,"$72,800/yr - $109,200/yr (from job description) · Full-time · Entry level",201-500 employees · Education Management,Remote,Remote
13,"Maine, United States",Hybrid,2 weeks ago,38 applicants,Full-time · Mid-Senior level,"501-1,000 employees · Technology, Information and Internet",NaN,Maine
14,"McLean, VA",Hybrid,2 weeks ago,113 applicants,"$58,400/yr - $133,000/yr (from job description) · Full-time","10,001+ employees · IT Services and IT Consulting",McLean,Virginia


### Split Company Overview
New columns:
- Company Size
- Industry

In [519]:
job_df['Company Overview'].str.split(' · ').map(len).value_counts()

2    844
1     41
Name: Company Overview, dtype: int64

In [520]:
list(set(job_df['Company Overview'].str.split(' · ').values.sum()))

['51-200 employees',
 'Insurance',
 'Truck Transportation',
 'Wholesale Building Materials',
 'Data Infrastructure and Analytics',
 'Telecommunications',
 'Beverage Manufacturing',
 'Airlines and Aviation',
 'Business Consulting and Services',
 'Internet Publishing',
 '5,001-10,000 employees',
 'Staffing and Recruiting',
 '10,001+ employees',
 'Education Administration Programs',
 'Oil and Gas',
 'Advertising Services',
 'Information Technology &amp; Services',
 'Biotechnology',
 'Higher Education',
 'Human Resources Services',
 '201-500 employees',
 'Retail Apparel and Fashion',
 'Industrial Machinery Manufacturing',
 'Defense &amp; Space',
 '1,001-5,000 employees',
 '1-10 employees',
 'Aviation and Aerospace Component Manufacturing',
 'Civil Engineering',
 'Food Production',
 'Research',
 'Wellness and Fitness Services',
 'Broadcast Media Production and Distribution',
 'Online Audio and Video Media',
 'Real Estate',
 'Investment Banking',
 '11-50 employees',
 'Pharmaceutical Manufact

#### Remove invalid Company Overview values

In [521]:
job_df.loc[~job_df['Company Overview'].str.contains('employees')]

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State
534,3525735059,https://www.linkedin.com/jobs/view/3525735059,Data Engineer,NaN,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7","Dallas, TX",Hybrid,1 week ago,92 applicants,Full-time · Mid-Senior level,"See how you compare to 92 applicants.<span class=""white-space-pre""> </span><a class=""app-aware-link "" target=""_self"" href=""https://www.linkedin.com/premium/products/?family=JSS&amp;upsellOrderOrigin=premium_job_details_summary_card&amp;utype=job"" data-test-app-aware-link="""">Try Premium for free</a>",Dallas,Texas
603,3525016432,https://www.linkedin.com/jobs/view/3525016432,Project Manager,NaN,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7","Stuart, FL",Hybrid,2 weeks ago,44 applicants,Full-time,Greg M. is hiring for this job,Stuart,Florida


In [522]:
job_df.loc[~job_df['Company Overview'].str.contains('employees'), 'Company Overview'] = 'Unknown' # replace invalid Company Overview with Unknown
job_df[job_df['Company Overview'] == 'Unknown'] # recheck

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State
534,3525735059,https://www.linkedin.com/jobs/view/3525735059,Data Engineer,NaN,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7","Dallas, TX",Hybrid,1 week ago,92 applicants,Full-time · Mid-Senior level,Unknown,Dallas,Texas
603,3525016432,https://www.linkedin.com/jobs/view/3525016432,Project Manager,NaN,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7","Stuart, FL",Hybrid,2 weeks ago,44 applicants,Full-time,Unknown,Stuart,Florida


#### Split method 1: regex

In [523]:
# seq = '1-10 employees'
# seq = '5,001-10,000 employees'
# seq = '10,001+ employees · Entertainment Providers'
# seq = '201-500 employees · Online Audio and Video Media'
# pattern = r"(\d+,?\d*\+?[\-?\d+,?\d+]* employees)( · )?(.+)*"
# re.search(pattern, seq)

In [524]:
pattern = r"(\d+,?\d*\+?[\-?\d+,?\d+]* employees)( · )?(.+)*"

for idx, row in job_df.iterrows():
    seq = row['Company Overview']
    company_overview = re.search(pattern, seq)
    job_df.at[idx, 'Company Size'] = company_overview.group(1) if company_overview is not None else 'Unknown'
    job_df.at[idx, 'Industry'] = company_overview.group(3) if company_overview is not None else 'Unknown'

job_df.iloc[:10, 5:]

,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Company Size,Industry
0,"San Jose, CA",Hybrid,2 weeks ago,198 applicants,Full-time,"10,001+ employees · Entertainment Providers",San Jose,California,"10,001+ employees",Entertainment Providers
1,United States,Remote,2 weeks ago,Over 200 applicants,Full-time · Mid-Senior level,"10,001+ employees · Manufacturing",Remote,Remote,"10,001+ employees",Manufacturing
2,"Washington, DC",Remote,NaN,NaN,$16/hr - $21/hr (from job description) · Internship · Internship,201-500 employees · Online Audio and Video Media,Remote,Remote,201-500 employees,Online Audio and Video Media
3,"Milford, MA",On-site,1 week ago,Over 200 applicants,Internship · Internship,"5,001-10,000 employees · Biotechnology Research",Milford,Massachusetts,"5,001-10,000 employees",Biotechnology Research
4,"Kennett Square, PA",On-site,2 weeks ago,104 applicants,Full-time · Entry level,"501-1,000 employees · Financial Services",Kennett Square,Pennsylvania,"501-1,000 employees",Financial Services
5,"Basking Ridge, NJ",NaN,2 days ago,25 applicants,Full-time,"1,001-5,000 employees · Pharmaceutical Manufacturing",Basking Ridge,New Jersey,"1,001-5,000 employees",Pharmaceutical Manufacturing
6,"Chicago, IL",Hybrid,2 weeks ago,Over 200 applicants,"$85,000/yr - $130,000/yr · Full-time · Mid-Senior level",11-50 employees · Staffing and Recruiting,Chicago,Illinois,11-50 employees,Staffing and Recruiting
7,"Washington, DC",Hybrid,2 weeks ago,Over 200 applicants,Full-time · Associate,51-200 employees · Defense and Space Manufacturing,None,District of Columbia,51-200 employees,Defense and Space Manufacturing
8,"McLean, VA",Hybrid,1 week ago,67 applicants,Full-time · Mid-Senior level,"1,001-5,000 employees · Staffing and Recruiting",McLean,Virginia,"1,001-5,000 employees",Staffing and Recruiting
9,"Indianola, PA",Hybrid,2 weeks ago,43 applicants,"$120,000/yr · Contract · Mid-Senior level","10,001+ employees · Chemical Manufacturing",Indianola,Pennsylvania,"10,001+ employees",Chemical Manufacturing


#### Split method 2: .split()

In [525]:
# mask = job_df[job_df['Company Overview'].str.contains('employees')]
# job_df[['Company Size', 'Industry']] = job_df['Company Overview'].str.split(' · ', expand=True)
# job_df.head()

*Note: This method only works when Company size is available in Company Overview*

### Split Job Overview

- Salary
- Workplace Type
- Level of Expertise

#### Create Salary Column

In [526]:
# salary_pattern = r"\$(\d)+.*\/(yr|hr)"
# seq = '$50,000/yr - $75,000/yr (from job description) · Full-time · Associate'
# seq = '$16.74/hr - $21.75/hr'
# seq = '$20/hr - $23/hr'
# seq = '$40/hr - $45/hr'
# result = re.search(salary_pattern, seq)
# result.group()

In [527]:
salary_pattern = r"\$(\d)+.*\/(yr|hr)"

for idx, row in job_df.iterrows():
    job_overview = row['Job Overview']
    try:
        salary = re.search(salary_pattern, job_overview)
        job_df.at[idx, 'Salary'] = salary.group() if salary is not None else 'Unknown'
    except NoSuchElementException:
        pass

job_df.head(5)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Company Size,Industry,Salary
0,3520044004,https://www.linkedin.com/jobs/view/3520044004,"Data Scientist, Research",TikTok,https://media.licdn.com/dms/image/C510BAQGCdThXIss7UQ/company-logo_100_100/0/1539940587971?e=1687996800&v=beta&t=1AyKLKEh0SiRwHFVslSVLJ-5LaOWsCzFqZtQ9ZvHi1o,"San Jose, CA",Hybrid,2 weeks ago,198 applicants,Full-time,"10,001+ employees · Entertainment Providers",San Jose,California,"10,001+ employees",Entertainment Providers,Unknown
1,3523743282,https://www.linkedin.com/jobs/view/3523743282,Global Data Scientist,Kimberly-Clark,https://media.licdn.com/dms/image/C560BAQFahtjOdf_ETQ/company-logo_100_100/0/1542208571146?e=1687996800&v=beta&t=y-aRHU6gnrNyr6nMPswJJHlWEoEOjZyHL-a1Qs5MPFY,United States,Remote,2 weeks ago,Over 200 applicants,Full-time · Mid-Senior level,"10,001+ employees · Manufacturing",Remote,Remote,"10,001+ employees",Manufacturing,Unknown
2,3520459813,https://www.linkedin.com/jobs/view/3520459813,Data Analytics Intern (Summer 2023),Industry Dive,https://media.licdn.com/dms/image/C4E0BAQEAkpLAgFUtpA/company-logo_100_100/0/1520994058492?e=1687996800&v=beta&t=wkHpT8VLjHRZryS_VDdpboOGmMkNRTNhMqVoDrCC2_M,"Washington, DC",Remote,NaN,NaN,$16/hr - $21/hr (from job description) · Internship · Internship,201-500 employees · Online Audio and Video Media,Remote,Remote,201-500 employees,Online Audio and Video Media,$16/hr - $21/hr
3,3525723554,https://www.linkedin.com/jobs/view/3525723554,Data Scientist Solution Specialist- IT Internship,Waters Corporation,https://media.licdn.com/dms/image/C560BAQHFDhBFVWfhzg/company-logo_100_100/0/1656651227123?e=1687996800&v=beta&t=YmLFUIyNIljy5hv0bST7P3wzrzdVBRln9qUpFAufG0c,"Milford, MA",On-site,1 week ago,Over 200 applicants,Internship · Internship,"5,001-10,000 employees · Biotechnology Research",Milford,Massachusetts,"5,001-10,000 employees",Biotechnology Research,Unknown
4,3527821880,https://www.linkedin.com/jobs/view/3527821880,Data Engineer,Chatham Financial,https://media.licdn.com/dms/image/C4D0BAQFPJJtAqKZSKA/company-logo_100_100/0/1566565993951?e=1687996800&v=beta&t=w4VlO0akxyqbbvH7Io6cMb3i8qShWG84zirwSGF-rDM,"Kennett Square, PA",On-site,2 weeks ago,104 applicants,Full-time · Entry level,"501-1,000 employees · Financial Services",Kennett Square,Pennsylvania,"501-1,000 employees",Financial Services,Unknown


#### Create Workplace Type & Level of Expertise columns

In [528]:
tmp = job_df.copy()
tmp = tmp['Job Overview'].str.replace(' (from job description)', '', regex=False)
tmp = tmp.str.replace(r'\$(\d)+.*\/(yr|hr)', '', regex=True).str.strip(' · ')
list(set(tmp.str.split(' · ').values.sum()))

['Executive',
 'Director',
 'Entry level',
 'Full-time',
 '$6,597/month - $7,431/month',
 'Temporary',
 'Internship',
 'Associate',
 'Contract',
 '$5,250/month - $5,583/month',
 '$5,380/month - $7,372/month',
 'Part-time',
 'Mid-Senior level']

In [529]:
contract_type_list = ['Temporary', 'Part-time', 'Full-time', 'Internship', 'Contract']
contract_type_pattern = '|'.join(contract_type_list)

for idx, row in job_df.iterrows():
    job_overview = row['Job Overview']
    try:
        contract_type = re.search(contract_type_pattern, job_overview)
        job_df.at[idx, 'Contract Type'] = contract_type.group() if contract_type is not None else 'Unknown'
    except NoSuchElementException:
        pass
job_df.head(5)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Company Size,Industry,Salary,Contract Type
0,3520044004,https://www.linkedin.com/jobs/view/3520044004,"Data Scientist, Research",TikTok,https://media.licdn.com/dms/image/C510BAQGCdThXIss7UQ/company-logo_100_100/0/1539940587971?e=1687996800&v=beta&t=1AyKLKEh0SiRwHFVslSVLJ-5LaOWsCzFqZtQ9ZvHi1o,"San Jose, CA",Hybrid,2 weeks ago,198 applicants,Full-time,"10,001+ employees · Entertainment Providers",San Jose,California,"10,001+ employees",Entertainment Providers,Unknown,Full-time
1,3523743282,https://www.linkedin.com/jobs/view/3523743282,Global Data Scientist,Kimberly-Clark,https://media.licdn.com/dms/image/C560BAQFahtjOdf_ETQ/company-logo_100_100/0/1542208571146?e=1687996800&v=beta&t=y-aRHU6gnrNyr6nMPswJJHlWEoEOjZyHL-a1Qs5MPFY,United States,Remote,2 weeks ago,Over 200 applicants,Full-time · Mid-Senior level,"10,001+ employees · Manufacturing",Remote,Remote,"10,001+ employees",Manufacturing,Unknown,Full-time
2,3520459813,https://www.linkedin.com/jobs/view/3520459813,Data Analytics Intern (Summer 2023),Industry Dive,https://media.licdn.com/dms/image/C4E0BAQEAkpLAgFUtpA/company-logo_100_100/0/1520994058492?e=1687996800&v=beta&t=wkHpT8VLjHRZryS_VDdpboOGmMkNRTNhMqVoDrCC2_M,"Washington, DC",Remote,NaN,NaN,$16/hr - $21/hr (from job description) · Internship · Internship,201-500 employees · Online Audio and Video Media,Remote,Remote,201-500 employees,Online Audio and Video Media,$16/hr - $21/hr,Internship
3,3525723554,https://www.linkedin.com/jobs/view/3525723554,Data Scientist Solution Specialist- IT Internship,Waters Corporation,https://media.licdn.com/dms/image/C560BAQHFDhBFVWfhzg/company-logo_100_100/0/1656651227123?e=1687996800&v=beta&t=YmLFUIyNIljy5hv0bST7P3wzrzdVBRln9qUpFAufG0c,"Milford, MA",On-site,1 week ago,Over 200 applicants,Internship · Internship,"5,001-10,000 employees · Biotechnology Research",Milford,Massachusetts,"5,001-10,000 employees",Biotechnology Research,Unknown,Internship
4,3527821880,https://www.linkedin.com/jobs/view/3527821880,Data Engineer,Chatham Financial,https://media.licdn.com/dms/image/C4D0BAQFPJJtAqKZSKA/company-logo_100_100/0/1566565993951?e=1687996800&v=beta&t=w4VlO0akxyqbbvH7Io6cMb3i8qShWG84zirwSGF-rDM,"Kennett Square, PA",On-site,2 weeks ago,104 applicants,Full-time · Entry level,"501-1,000 employees · Financial Services",Kennett Square,Pennsylvania,"501-1,000 employees",Financial Services,Unknown,Full-time


In [530]:
exp_levels_list = ['Entry level', 'Junior', 'Mid-Senior level', 'Associate', 'Executive', 'Director']
exp_levels_pattern = '|'.join(exp_levels_list)

for idx, row in job_df.iterrows():
    job_overview = row['Job Overview']
    try:
        exp_level = re.search(exp_levels_pattern, job_overview)
        job_df.at[idx, 'Level of Expertise'] = exp_level.group() if exp_level is not None else 'Unknown'
    except NoSuchElementException:
        pass
job_df.head(5)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Company Size,Industry,Salary,Contract Type,Level of Expertise
0,3520044004,https://www.linkedin.com/jobs/view/3520044004,"Data Scientist, Research",TikTok,https://media.licdn.com/dms/image/C510BAQGCdThXIss7UQ/company-logo_100_100/0/1539940587971?e=1687996800&v=beta&t=1AyKLKEh0SiRwHFVslSVLJ-5LaOWsCzFqZtQ9ZvHi1o,"San Jose, CA",Hybrid,2 weeks ago,198 applicants,Full-time,"10,001+ employees · Entertainment Providers",San Jose,California,"10,001+ employees",Entertainment Providers,Unknown,Full-time,Unknown
1,3523743282,https://www.linkedin.com/jobs/view/3523743282,Global Data Scientist,Kimberly-Clark,https://media.licdn.com/dms/image/C560BAQFahtjOdf_ETQ/company-logo_100_100/0/1542208571146?e=1687996800&v=beta&t=y-aRHU6gnrNyr6nMPswJJHlWEoEOjZyHL-a1Qs5MPFY,United States,Remote,2 weeks ago,Over 200 applicants,Full-time · Mid-Senior level,"10,001+ employees · Manufacturing",Remote,Remote,"10,001+ employees",Manufacturing,Unknown,Full-time,Mid-Senior level
2,3520459813,https://www.linkedin.com/jobs/view/3520459813,Data Analytics Intern (Summer 2023),Industry Dive,https://media.licdn.com/dms/image/C4E0BAQEAkpLAgFUtpA/company-logo_100_100/0/1520994058492?e=1687996800&v=beta&t=wkHpT8VLjHRZryS_VDdpboOGmMkNRTNhMqVoDrCC2_M,"Washington, DC",Remote,NaN,NaN,$16/hr - $21/hr (from job description) · Internship · Internship,201-500 employees · Online Audio and Video Media,Remote,Remote,201-500 employees,Online Audio and Video Media,$16/hr - $21/hr,Internship,Unknown
3,3525723554,https://www.linkedin.com/jobs/view/3525723554,Data Scientist Solution Specialist- IT Internship,Waters Corporation,https://media.licdn.com/dms/image/C560BAQHFDhBFVWfhzg/company-logo_100_100/0/1656651227123?e=1687996800&v=beta&t=YmLFUIyNIljy5hv0bST7P3wzrzdVBRln9qUpFAufG0c,"Milford, MA",On-site,1 week ago,Over 200 applicants,Internship · Internship,"5,001-10,000 employees · Biotechnology Research",Milford,Massachusetts,"5,001-10,000 employees",Biotechnology Research,Unknown,Internship,Unknown
4,3527821880,https://www.linkedin.com/jobs/view/3527821880,Data Engineer,Chatham Financial,https://media.licdn.com/dms/image/C4D0BAQFPJJtAqKZSKA/company-logo_100_100/0/1566565993951?e=1687996800&v=beta&t=w4VlO0akxyqbbvH7Io6cMb3i8qShWG84zirwSGF-rDM,"Kennett Square, PA",On-site,2 weeks ago,104 applicants,Full-time · Entry level,"501-1,000 employees · Financial Services",Kennett Square,Pennsylvania,"501-1,000 employees",Financial Services,Unknown,Full-time,Entry level


### Replace all None and emptry strings with Unknown

In [531]:
job_df = job_df.replace('', np.nan).fillna('Unknown')

# EDA

In [532]:
cols = ['Job URL', 'Name', 'Company', 'Location', 'Location_City', 'Location_State',
       'Workplace Type', 'Time Posted', 'Applicants Count',
        'Salary', 'Contract Type', 'Level of Expertise',
       'Company Size', 'Industry']
df = job_df[cols]

In [533]:
df.head(10)

,Job URL,Name,Company,Location,Location_City,Location_State,Workplace Type,Time Posted,Applicants Count,Salary,Contract Type,Level of Expertise,Company Size,Industry
0,https://www.linkedin.com/jobs/view/3520044004,"Data Scientist, Research",TikTok,"San Jose, CA",San Jose,California,Hybrid,2 weeks ago,198 applicants,Unknown,Full-time,Unknown,"10,001+ employees",Entertainment Providers
1,https://www.linkedin.com/jobs/view/3523743282,Global Data Scientist,Kimberly-Clark,United States,Remote,Remote,Remote,2 weeks ago,Over 200 applicants,Unknown,Full-time,Mid-Senior level,"10,001+ employees",Manufacturing
2,https://www.linkedin.com/jobs/view/3520459813,Data Analytics Intern (Summer 2023),Industry Dive,"Washington, DC",Remote,Remote,Remote,Unknown,Unknown,$16/hr - $21/hr,Internship,Unknown,201-500 employees,Online Audio and Video Media
3,https://www.linkedin.com/jobs/view/3525723554,Data Scientist Solution Specialist- IT Internship,Waters Corporation,"Milford, MA",Milford,Massachusetts,On-site,1 week ago,Over 200 applicants,Unknown,Internship,Unknown,"5,001-10,000 employees",Biotechnology Research
4,https://www.linkedin.com/jobs/view/3527821880,Data Engineer,Chatham Financial,"Kennett Square, PA",Kennett Square,Pennsylvania,On-site,2 weeks ago,104 applicants,Unknown,Full-time,Entry level,"501-1,000 employees",Financial Services
5,https://www.linkedin.com/jobs/view/3545964641,"Programmer, Data Analysis, Epidemiology Analytics","Daiichi Sankyo, Inc.","Basking Ridge, NJ",Basking Ridge,New Jersey,Unknown,2 days ago,25 applicants,Unknown,Full-time,Unknown,"1,001-5,000 employees",Pharmaceutical Manufacturing
6,https://www.linkedin.com/jobs/view/3520729278,Investment Data Analyst,Westbourne Partners,"Chicago, IL",Chicago,Illinois,Hybrid,2 weeks ago,Over 200 applicants,"$85,000/yr - $130,000/yr",Full-time,Mid-Senior level,11-50 employees,Staffing and Recruiting
7,https://www.linkedin.com/jobs/view/3516637194,"Data Scientist, DC",Rhombus Power Inc.,"Washington, DC",Unknown,District of Columbia,Hybrid,2 weeks ago,Over 200 applicants,Unknown,Full-time,Associate,51-200 employees,Defense and Space Manufacturing
8,https://www.linkedin.com/jobs/view/3527065467,Data Operations Engineer / Business Operations Analyst,US Tech Solutions,"McLean, VA",McLean,Virginia,Hybrid,1 week ago,67 applicants,Unknown,Full-time,Mid-Senior level,"1,001-5,000 employees",Staffing and Recruiting
9,https://www.linkedin.com/jobs/view/3522212905,Medical Software Quality Engineer - $120k/yr,Bayer,"Indianola, PA",Indianola,Pennsylvania,Hybrid,2 weeks ago,43 applicants,"$120,000/yr",Contract,Mid-Senior level,"10,001+ employees",Chemical Manufacturing


In [534]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 885 entries, 0 to 884
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Job URL             885 non-null    object
 1   Name                885 non-null    object
 2   Company             885 non-null    object
 3   Location            885 non-null    object
 4   Location_City       885 non-null    object
 5   Location_State      885 non-null    object
 6   Workplace Type      885 non-null    object
 7   Time Posted         885 non-null    object
 8   Applicants Count    885 non-null    object
 9   Salary              885 non-null    object
 10  Contract Type       885 non-null    object
 11  Level of Expertise  885 non-null    object
 12  Company Size        885 non-null    object
 13  Industry            885 non-null    object
dtypes: object(14)
memory usage: 136.0+ KB


#### Filter out Jobs relevant to Data Analyst

In [538]:
# r"data|bi|analyst|analytics|machine learning|ml|ai|intelligence"
da_df = df[df.Name.str.contains(r"data analyst|analyst|analytics", case=False, regex=True)]
display(da_df.head())
print(f'Found {da_df.shape[0]} jobs relavant to data analyst')

,Job URL,Name,Company,Location,Location_City,Location_State,Workplace Type,Time Posted,Applicants Count,Salary,Contract Type,Level of Expertise,Company Size,Industry
2,https://www.linkedin.com/jobs/view/3520459813,Data Analytics Intern (Summer 2023),Industry Dive,"Washington, DC",Remote,Remote,Remote,Unknown,Unknown,$16/hr - $21/hr,Internship,Unknown,201-500 employees,Online Audio and Video Media
5,https://www.linkedin.com/jobs/view/3545964641,"Programmer, Data Analysis, Epidemiology Analytics","Daiichi Sankyo, Inc.","Basking Ridge, NJ",Basking Ridge,New Jersey,Unknown,2 days ago,25 applicants,Unknown,Full-time,Unknown,"1,001-5,000 employees",Pharmaceutical Manufacturing
6,https://www.linkedin.com/jobs/view/3520729278,Investment Data Analyst,Westbourne Partners,"Chicago, IL",Chicago,Illinois,Hybrid,2 weeks ago,Over 200 applicants,"$85,000/yr - $130,000/yr",Full-time,Mid-Senior level,11-50 employees,Staffing and Recruiting
8,https://www.linkedin.com/jobs/view/3527065467,Data Operations Engineer / Business Operations Analyst,US Tech Solutions,"McLean, VA",McLean,Virginia,Hybrid,1 week ago,67 applicants,Unknown,Full-time,Mid-Senior level,"1,001-5,000 employees",Staffing and Recruiting
12,https://www.linkedin.com/jobs/view/3532901532,Analyst,TNTP,United States,Remote,Remote,Remote,1 week ago,Over 200 applicants,"$72,800/yr - $109,200/yr",Full-time,Entry level,201-500 employees,Education Management


Found 237 jobs relavant to data analyst


#### Entry level jobs

In [539]:
da_df[da_df['Level of Expertise']=='Entry level']

,Job URL,Name,Company,Location,Location_City,Location_State,Workplace Type,Time Posted,Applicants Count,Salary,Contract Type,Level of Expertise,Company Size,Industry
12,https://www.linkedin.com/jobs/view/3532901532,Analyst,TNTP,United States,Remote,Remote,Remote,1 week ago,Over 200 applicants,"$72,800/yr - $109,200/yr",Full-time,Entry level,201-500 employees,Education Management
27,https://www.linkedin.com/jobs/view/3542549318,Staff HR Data Analyst,Ridgeline,"Incline Village, NV",Incline Village,Nevada,Hybrid,2 days ago,4 applicants,"$130,000/yr - $159,000/yr",Full-time,Entry level,201-500 employees,Software Development
37,https://www.linkedin.com/jobs/view/3527860425,Analyst,Greystone,"New York, NY",New York,New York,On-site,2 weeks ago,Over 200 applicants,"$80,000/yr - $90,000/yr",Full-time,Entry level,"1,001-5,000 employees",Financial Services
52,https://www.linkedin.com/jobs/view/3531482679,Data Analyst,Baker Hughes,"Stafford, TX",Stafford,Texas,On-site,14 hours ago,Over 200 applicants,Unknown,Full-time,Entry level,"10,001+ employees",Oil and Gas
89,https://www.linkedin.com/jobs/view/3539667050,Quality Data Analyst,Zobility,"Toledo, OH",Toledo,Ohio,Unknown,2 days ago,1 applicant,Unknown,Contract,Entry level,201-500 employees,Staffing and Recruiting
92,https://www.linkedin.com/jobs/view/3529465223,Data Integration Analyst,Huxley,"Boston, MA",Remote,Remote,Remote,2 weeks ago,Over 200 applicants,Unknown,Full-time,Entry level,201-500 employees,Staffing and Recruiting
99,https://www.linkedin.com/jobs/view/3529172289,Junior Analyst,Scientific Research Corporation,"Warner Robins, GA",Warner Robins,Georgia,On-site,1 week ago,29 applicants,Unknown,Full-time,Entry level,"1,001-5,000 employees",Defense and Space Manufacturing
110,https://www.linkedin.com/jobs/view/3521902207,Data Engineer Analyst,Guidehouse,"Huntsville, AL",Huntsville,Alabama,On-site,2 weeks ago,36 applicants,Unknown,Full-time,Entry level,"10,001+ employees",Business Consulting and Services
121,https://www.linkedin.com/jobs/view/3526062290,Power Platform Data Analyst,AccruePartners,"Iselin, NJ",Iselin,New Jersey,On-site,1 week ago,45 applicants,Unknown,Contract,Entry level,201-500 employees,Staffing and Recruiting
122,https://www.linkedin.com/jobs/view/3530024400,Data Analytics (Hybrid),U.S. Bank,"Milwaukee, WI",Milwaukee,Wisconsin,Unknown,2 weeks ago,114 applicants,Unknown,Full-time,Entry level,"10,001+ employees",Banking
